<a href="https://colab.research.google.com/github/asrjy/ldrl/blob/main/Chapter%206%20%20-%20Tabular%20Learning%20and%20the%20Bellman%20Equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tabular Learning and the Bellman Equation


The value of a state has been defined as, the expected total reward (discounted or not) from the state. 
ie., 

value = (sum of all rewards * discount factor ^ t) for all t

the optimal policy is the one which gives the most reward as possible, by extension the best value of each state. 

such simple environments with an obvious policy are not interesting and not generally seen in real life. for the interesting environments, optimal policies are hard to devise and even harder to prove their optimality. 

its not always good to be "greedy" ie., taking the action with the highest reward. 


### The Bellman equation of optimality

the approach may look similar to the greedy approach but the key difference is that we not only look at the immediate reward, but also the value of the state that lets us know about the future rewards. 

so to get the expected value, 

E = for all states(probability of each action possible at that state * (reward for that action + (gamma * value of the state we end up in)))

using this, the Bellman Optimality Equation can be defined for a general case (both stochastic and deterministic) as

![image.png](https://static.packt-cdn.com/products/9781838826994/graphics/Images/B14854_05_010.png)

P (a, i -> j) means probabilty of action a, issued in state i, ending up in state j.

the equation is recursive in nature (the value of the state is defined using the value of target states), we pretend we know the values by randomly initializing them. 

### The value of the equation

Q(s, a) is the total reward we can get by executing action a in state s, it can be obtained using V(s). This is the Q in Q-learning whose whole objective is to get values of Q for every pair of state and action.

it can be defined as 

sum of for each state s' in S (probability of ending up in state s' by performing action a in state s * (reward for action a in state s + (gamma * value of state s')))

![image.png](https://static.packt-cdn.com/products/9781838826994/graphics/Images/B14854_05_012.png)

value of state s can also be expressed using Q function as 

![value using q function](https://static.packt-cdn.com/products/9781838826994/graphics/Images/B14854_05_013.png)

and we can express Q function recursively which will be used extensively in Deep Q Learning as

![q function recursive](https://static.packt-cdn.com/products/9781838826994/graphics/Images/B14854_05_014.png)

Q values are much more convenient in practice than Value functions because it's simpler to make decisions about actions obtained from Q function than using value of state obtained from value function. at any given state, the agent just needs to calculate the q values of all available actions at that state and pick the one with the maximum q value. to do the same with value functions, it needs to know the transitional probabilities before hand which is very rare in practice, so we need to estimate the transitional probablities of every action and state pair. 




### The Value Iteration method

the above algorithm works well, but if there are any loops in the transitions, it leads to an infinite pool. 

in this case, we use value iteration algorithm. it works as follows:

- initialize values of all states to some initial value, usually 0.
- for every state s, perform the bellman update. 
![value iteration for value function](https://static.packt-cdn.com/products/9781838826994/graphics/Images/B14854_05_023.png)
- repeat the above step for a large number of times or until the change becomes too small. 

the same algorithm can be used for q function as well. 
![value iteration for q function](https://static.packt-cdn.com/products/9781838826994/graphics/Images/B14854_05_024.png)

the issue with this is that, the state space needs to be discrete and small enough to perform multiple iterations over all states. this is not an issue for small environments like frozenlake, but an issue with environments like cartpole, where the observation is four float values. we could split the space into bins as they dont extend to infinite in this case, but choosing the right bin size is another issue. this could be fixed by using neural networks in q learning. 

another issue with the value iteration approach is that we rarely know the transition probabilities of actions and rewards. the way gym works is, we observe the state, perform an action and only then we get the next observation and reward for the action performed. the only way to get the transition probabilties before hand is to cheat and look at the environment's code. 

so the only way to overcome this issue is to use the agent's history ie., keep track of what actions at what states gave how much reward. 

### Value Iteration in practice

data stored:
- reward table that stores state, action, target state
- transitions table. if at state 0, we execute action 1 ten times where three times it takes us to state 4 and the rest of the times to state 5, in the table at the location (0,1), we store the dictionary {4:3, 5:7}. this denotes we go to state 4 after performing action 1 3 times and we go to state 5 after performing action 1 7 more times. 
- value table that stores the state and the calculated value of this state. 

we play 100 random steps from the environment updating the reward and transition table. then we perform value iteration loop over all states updating the value table. then we play several full episodes to check for improved values using value table. if average reward is above the 0.8 boundary for those test episodes, we stop training. we update reward and transition tables even during the testing phase. 


In [9]:
!pip install tensorboardX

In [10]:
import gym
import collections
from tensorboardX import SummaryWriter
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20

In [ ]:
class Agent:
  """
  This class keeps our tables and contains functions that will be used in the training loop
  """
  def __init__(self):
    """
    Obtaining the initial observation using reset, creating the environment that will be 
    used for the data samples, define tables for rewards, transitions and value tables
    """
    self.env = gym.make(ENV_NAME)
    self.state = self.env.reset()
    self.rewards = collections.defaultdict(float)
    self.transits = collections.defaultdict(collections.Counter)
    self.values = collections.defaultdict(float)

  def play_n_random_steps(self, count):
    """
    This function gathers random experience from the environment and update the reward and transition
    tables. cross-entropy can only learn on full episodes, where as value iteration can learn on individual
    steps/actions by remembering their outcomes. 
    """
    for _ in range(count):
      action = self.env.action_space.sample()
      new_state, reward, is_done, _ = self.env.step(action)
      self.rewards[(self.state, action, new_state)] = reward
      self.transits[(self.state, action)][new_state] += 1
      if is_done:
        self.state = self.env.reset()
      else:
        self.state = new_state

  def calculate_action_value(self, state, action):
    """
    calculates the value of action from state using the tables created in this class. this lets us select the
    best action to perform from the state and calculate new value of state in value iteration
    since the transition table is stored in the form of dict, we can use this to get the probabilities of 
    ending up in a state. 
    these probabilities are used in the bellman equation to get the q function which is the sum of rewards and 
    discounted value of target state multiplied with probablities. 
    """
    target_counts = self.transits[(state, action)]
    total = sum(target_counts.values())
    action_value = 0.0
    for tgt_state, count in target_counts.items():
      reward = self.rewards[(state, action, tgt_state)]
      val = reward + GAMMA * self.values[tgt_state]
      action_value += (count/total) * val
    return action_value

  def select_action(self, state):
    """
    this uses the calculate_action_value() to make decision about the best action to take from a given state. 
    it iterates over all possible actions and picks the action with highest value    
    """
    best_action, best_value = None, None
    for action in range(self.env.action_space.n):
      action_value = self.calculate_action_value(state, action)
      if best_value is None or best_value < action_value:
        best_value = action_value
        best_action = action
    return best_action
  
  def play_episode(self, env):
    """
    this function uses select_action() to find the best action to take and plays one episode with the 
    environment. this is useful to play test episodes where we dont want to mess with current state of main
    environment. 
    """
    total_reward = 0.0
    state = env.reset()
    while True:
      action = self.select_action(state)
      new_state, reward, is_done, _ = env.step(action)
      self.rewards[(state, action, new_state)] = reward
      self.transits[(state, action)][new_state] += 1
      total_reward += reward
      if is_done:
        break
      state = new_state
    return total_reward 
  
  def value_iteration(self):
    """
    we loop over all states in the environment, for every state, calculate the values of states reachable 
    from it, then update value of current state with maximum value of the action available from the state
    """
    for state in range(self.env.observation_space.n):
      state_values = [
                      self.calculate_action_value(state, action)
                      for action in range(self.env.action_space.n)
      ]
      self.values[state] = max(state_values)

In [ ]:
if __name__ == "__main__":
  test_env = gym.make(ENV_NAME)
  agent = Agent()
  writer = SummaryWriter(comment="-v-iteration")
  iter_no = 0
  best_reward = 0.0
  while True:
    iter_no += 1
    # playing 100 random steps to fill transition and reward tables, then running value iteration
    # over all states. 
    agent.play_n_random_steps(100)
    agent.value_iteration()
    # test episodes using the value table obtained so far. 
    reward = 0.0
    for _ in range(TEST_EPISODES):
      reward += agent.play_episode(test_env)
      reward /= TEST_EPISODES
      writer.add_scalar("reward", reward, iter_no)
      if reward > best_reward:
        print(f"Best reward updated {best_reward:.3f} -> {reward:.3f}")
        best_reward = reward
      if reward > 0.80:
        print(f"Solved in {iter_no} iteration")
        break
  writer.close()


### Q-Learning for Frozen Lake

In this, instead of storing the value of the state, we need to store the values of q function, but value of state has only one key, the state, but the q function has two keys, state and action. 

we also don't need the calculate_action_value() function anymore. 

we also rewrite the value_iteration() function, as we don't just loop around states like in value function, we iterate over every state action pair. 

In [11]:
class Agent:
  """
  This class keeps our tables and contains functions that will be used in the training loop
  """
  def __init__(self):
    """
    Obtaining the initial observation using reset, creating the environment that will be 
    used for the data samples, define tables for rewards, transitions and value tables
    """
    self.env = gym.make(ENV_NAME)
    self.state = self.env.reset()
    self.rewards = collections.defaultdict(float)
    self.transits = collections.defaultdict(collections.Counter)
    self.values = collections.defaultdict(float)

  def play_n_random_steps(self, count):
    """
    This function gathers random experience from the environment and update the reward and transition
    tables. cross-entropy can only learn on full episodes, where as value iteration can learn on individual
    steps/actions by remembering their outcomes. 
    """
    for _ in range(count):
      action = self.env.action_space.sample()
      new_state, reward, is_done, _ = self.env.step(action)
      self.rewards[(self.state, action, new_state)] = reward
      self.transits[(self.state, action)][new_state] += 1
      if is_done:
        self.state = self.env.reset()
      else:
        self.state = new_state

  def calculate_action_value(self, state, action):
    """
    calculates the value of action from state using the tables created in this class. this lets us select the
    best action to perform from the state and calculate new value of state in value iteration
    since the transition table is stored in the form of dict, we can use this to get the probabilities of 
    ending up in a state. 
    these probabilities are used in the bellman equation to get the q function which is the sum of rewards and 
    discounted value of target state multiplied with probablities. 
    """
    target_counts = self.transits[(state, action)]
    total = sum(target_counts.values())
    action_value = 0.0
    for tgt_state, count in target_counts.items():
      reward = self.rewards[(state, action, tgt_state)]
      val = reward + GAMMA * self.values[tgt_state]
      action_value += (count/total) * val
    return action_value

  def select_action(self, state):
    """
    this uses the calculate_action_value() to make decision about the best action to take from a given state. 
    it iterates over all possible actions and picks the action with highest value    
    """
    best_action, best_value = None, None
    for action in range(self.env.action_space.n):
      action_value = self.values[(state, action)]
      if best_value is None or best_value < action_value:
        best_value = action_value
        best_action = action
    return best_action
  
  def play_episode(self, env):
    """
    this function uses select_action() to find the best action to take and plays one episode with the 
    environment. this is useful to play test episodes where we dont want to mess with current state of main
    environment. 
    """
    total_reward = 0.0
    state = env.reset()
    while True:
      action = self.select_action(state)
      new_state, reward, is_done, _ = env.step(action)
      self.rewards[(state, action, new_state)] = reward
      self.transits[(state, action)][new_state] += 1
      total_reward += reward
      if is_done:
        break
      state = new_state
    return total_reward 
  
  def value_iteration(self):
    """
    for every state and action pair, it calculates the value of this action
    """
    for state in range(self.env.observation_space.n):
      for action in range(self.env.action_space.n):
        action_value = 0.0
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        for tgt_state, count in target_counts.items():
          key = (state, action, tgt_state)
          reward = self.rewards[key]
          best_action = self.select_action(tgt_state)
          val = reward + GAMMA * self.values[(tgt_state, best_action)]
          action_value += (count/total) * val
        self.values[(state, action)] = action_value
          

In [ ]:
if __name__ == "__main__":
  test_env = gym.make(ENV_NAME)
  agent = Agent()
  writer = SummaryWriter(comment="-v-iteration")
  iter_no = 0
  best_reward = 0.0
  while True:
    iter_no += 1
    # playing 100 random steps to fill transition and reward tables, then running value iteration
    # over all states. 
    agent.play_n_random_steps(100)
    agent.value_iteration()
    # test episodes using the value table obtained so far. 
    reward = 0.0
    for _ in range(TEST_EPISODES):
      reward += agent.play_episode(test_env)
      reward /= TEST_EPISODES
      writer.add_scalar("reward", reward, iter_no)
      if reward > best_reward:
        print(f"Best reward updated {best_reward:.3f} -> {reward:.3f}")
        best_reward = reward
      if reward > 0.80:
        print(f"Solved in {iter_no} iteration")
        break
  writer.close()
